In [1]:
# загрузим библиотеки

import pandas as pd
import dash
from dash import Dash, dcc, html 
import plotly.express as px
from dash.dependencies import Input, Output
from datetime import datetime as dt
import numpy as np

In [2]:
# загрузим данные
data = pd.read_csv("./games.csv")

In [3]:
data

,Name,Platform,Year_of_Release,Genre,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,NaN,NaN,NaN
...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,NaN,NaN,NaN


In [4]:
# посмотрим на файл

# посмотрим на тип и количество данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Critic_Score     8137 non-null   float64
 5   User_Score       10015 non-null  object 
 6   Rating           9950 non-null   object 
dtypes: float64(2), object(5)
memory usage: 914.4+ KB


In [5]:
# посмотрим количество пропущенных значений
data.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Critic_Score       8582
User_Score         6704
Rating             6769
dtype: int64

In [6]:
# посмотрим количество уникальных значений
data.nunique()

Name               11562
Platform              31
Year_of_Release       39
Genre                 12
Critic_Score          82
User_Score            96
Rating                 8
dtype: int64

In [7]:
# посмотрим статистические данные числовых столбцов
data.describe()

,Year_of_Release,Critic_Score
count,16450.000000,8137.000000
mean,2006.487356,68.967679
std,5.878995,13.938165
min,1980.000000,13.000000
25%,2003.000000,60.000000
50%,2007.000000,71.000000
75%,2010.000000,79.000000
max,2020.000000,98.000000


In [8]:
# Из данных нужно исключить проекты ранее 2000 года и позднее 2022
data = data[(data["Year_of_Release"] >= 2000) & (data["Year_of_Release"] <= 2022) & (data["User_Score"] != "tbd")]

In [9]:
# исключим проекты, для которых имеются пропуски данных в любой из колонок
data = data.dropna()

In [10]:
# поменяем тип данные в Year_of_Release с float64 на формат года
data["Year_of_Release"] = pd.to_datetime(data["Year_of_Release"].astype(int), format="%Y").dt.year

In [11]:
# создадим Dash
app = dash.Dash()

# добавим списки возможные жанров, рейтинга, периодов и платформ
available_genre = data["Genre"].unique()
available_rating = data["Rating"].unique()
available_years = data["Year_of_Release"].unique()
available_platforms = data["Platform"].unique()

# визуальная концепция dash
app.layout = html.Div([
        # заголовок
        html.Div([
            html.H1("Состояние игровой индустрии"),
            html.P(
            "Анализ игровой индустрии с 2000 по 2022 год. "
            "Используйте фильтры, чтобы увидеть результат."
            )
        ], style = {
            "padding": "10px 5px"
        }),


        html.Div([
        # фильтр по жанрам
            html.Div([
                html.Label ("Жанры игр"),
                dcc.Dropdown( 
                    id = "crossfilter-genre",
                    options = [{"label": i, "value": i} for i in available_genre],
                    # значения жанров по умолчанию
                    value = ["Sports", "Strategy"],
                    # множественный выбор
                    multi = True
                )
            ],
            style = {"width": "33%", "display": "inline-block"}),
             
        # фильтр по рейтингам
            html.Div([
                html.Label("Рейтинги игр"),
                dcc.Dropdown(
                    id = "crossfilter-rating",
                    options = [{"label": i, "value": i} for i in available_rating],
                    # значения рейтингов по умолчанию
                    value = ["T", "E"],
                    multi = True
                )
            ],
            style = {"width": "33%", "display": "inline-block"}),

        # фильтр по датам
            html.Div([
                html.Label("Интервал годов выпуска"),
                dcc.DatePickerRange(
                    id = "yearpick",
                    display_format="YYYY",
                    month_format="YYYY",
                    start_date=dt.strptime("2000", "%Y").date(),
                    end_date=dt.strptime("2008", "%Y").date(),
                )
            ],
            style = {"width": "33%","float": "right", "display": "inline-block"})
            
    ], style = {
    "borderBottom": "thin lightgrey solid",
    "backgroundColor": "rgb(250, 250, 250)",
    "padding": "15px 5px"
     }),

# заготовка для интерактивного текста - результата фильтрации
html.Div(
    id = "textarea-output",
    style = {"width": "33%", "float": "left", "display": "inline-block"}
    ),
    
html.Div(
    id = "textarea-output2",
    style = {"width": "33%", "display": "inline-block"}
    ),
    
html.Div(
    id = "textarea-output3",
    style = {"width": "33%", "float": "right", "display": "inline-block"}
    ),  

# гистограмма
html.Div(
    dcc. Graph(id = "histogram"),
    style = {"width": "33%", "float": "left", "display": "inline-block"}
    ),

# диаграмма рассеяния (scatter plot)
html.Div(
    dcc.Graph(id = "scatter-plot"),
    style = {"width": "33%", "display": "inline-block"}
    ),

# диаграмма рассеяния (scatter plot)
html.Div(
    dcc.Graph(id = "bar-line"),
    style = {"width": "33%", "float": "right", "display": "inline-block"}
    ),

])

@app.callback(
    Output ("textarea-output", "children"), 
    Output ("textarea-output2", "children"), 
    Output ("textarea-output3", "children"), 
    [Input("crossfilter-genre", "value"), 
     Input ("crossfilter-rating", "value"), 
     Input ("yearpick", "start_date"),
    Input ("yearpick", "end_date")
    ])

def update_textare(genre, rating, start_date, end_date):
    filtered_data = data[
                        (data["Year_of_Release"].astype(str) >= start_date) &
                        (data["Year_of_Release"].astype(str) <= end_date) &
                        (data[ "Genre"].isin(genre)) &
                        (data["Rating"].isin(rating))]
    # результат фильтрации
    games_count = len(filtered_data.index)
    mean_mark = round(np.mean(filtered_data.User_Score.astype("float")),2)
    critic_mark = round(np.mean(filtered_data.Critic_Score.astype("float")),2)
    return f"Общее число игр {games_count}. ", f"Общая средняя оценка игроков {mean_mark}.", f"Общая средняя оценка критиков {critic_mark}."

@app.callback(
    Output ("histogram", "figure"), 
    [Input("crossfilter-genre", "value"), 
     Input ("crossfilter-rating", "value"), 
     Input ("yearpick", "start_date"),
    Input ("yearpick", "end_date")
    ])

def update_stacked_area(genre, rating, start_date, end_date):
    filtered_data = data[
                        (data["Year_of_Release"].astype(str) >= start_date) &
                        (data["Year_of_Release"].astype(str) <= end_date) &
                        (data[ "Genre"].isin(genre)) &
                        (data["Rating"].isin(rating))]
    # задаем график 
    figure = px.histogram(
        filtered_data, 
        x = "Year_of_Release",
        color = "Platform",
        labels = {
            "Year_of_Release": "Год релиза игр",
            "Rating": "Рейтинг"
        },
        title = "Выпуск игр по годам и платформам"
    )
    figure.update_layout(yaxis_title = "Кол-во")
    return figure

@app.callback(
    Output ("scatter-plot", "figure"), 
    [Input("crossfilter-genre", "value"), 
     Input ("crossfilter-rating", "value"), 
     Input ("yearpick", "start_date"),
    Input ("yearpick", "end_date")
    ])

def update_scatter_plot(genre, rating, start_date, end_date):
    filtered_data = data[
                        (data["Year_of_Release"].astype(str) >= start_date) &
                        (data["Year_of_Release"].astype(str) <= end_date) &
                        (data["Genre"].isin(genre)) &
                        (data["Rating"].isin(rating))]
    # задаем график
    figure = px.scatter(
        filtered_data,
        x = "User_Score",
        y = "Critic_Score",
        color = "Genre",
        labels = {
            "User_Score": "Оценка пользователей",
            "Critic_Score": "Оценка критиков",
            "Genre": "Жанр"
        },
        title = "Зависимость оценок от жанров"
    )
    return figure

@app.callback(
    Output ("bar-line", "figure"), 
    [Input("crossfilter-genre", "value"), 
     Input ("crossfilter-rating", "value"), 
     Input ("yearpick", "start_date"),
    Input ("yearpick", "end_date")
    ])

def update_bar_chart(genre, rating, start_date, end_date):
    filtered_data = data[
                        (data["Year_of_Release"].astype(str) >= start_date) &
                        (data["Year_of_Release"].astype(str) <= end_date) &
                        (data["Genre"].isin(genre)) &
                        (data["Rating"].isin(rating))]
    # задаем график
    figure = px.histogram(
        filtered_data,
        x = "Genre",
        color = "Rating",
        barmode = "group",
        histfunc="avg",
        labels = {            
            "Rating": "Рейтинг",
            "Genre": "Жанр"
        },
        title = "Возрастной рейтинг по жанрам"
    )
    figure.update_layout(yaxis_title = "Кол-во")
    return figure



app.run_server(debug=True) 

P.S. не поняла, что в График 6 задании значит "средний" возрастной рейтинг, так как он не в цифрах, а в буквах согласно https://www.esrb.org. Поэтому сделала так, как поняла.  